### 교차 검증
- 부족한 데이터셋 및 특정 데이터에 과대적합되는 문제를 해결하기 위한 방안
- 학습 데이터셋을 일정 크기의 데이터로 n개 분리 후, 1/n은 검증용, 나머지는 학습용으로 사용

(1) 모듈 로딩 및 데이터 준비 <hr>

In [2]:
import numpy as np
from sklearn.model_selection import KFold

x = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])

In [3]:
# KFold 인스턴스 생성 => 데이터를 2개로 분할해주는 객체
k_fold = KFold(n_splits=2)

In [4]:
# 데이터 분할
datasets = k_fold.split(x)  # <generator object _BaseKFold.split at 0x0000021756531900>

for train, test in datasets:
    print(train, test)      # 출력되는 숫자들은 해당하는 데이터의 행번호

[2 3] [0 1]
[0 1] [2 3]


In [5]:
### perch3.csv 파일 데이터 5등분
import pandas as pd

filename = '../DATA/perch3.csv'
perchDF = pd.read_csv(filename)
perchDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Weight  56 non-null     float64
 1   Length  56 non-null     float64
 2   Height  56 non-null     float64
 3   Width   56 non-null     float64
dtypes: float64(4)
memory usage: 1.9 KB


In [6]:
# perchDF => 5등분 (default 값)
fold_5 = KFold()

datasets = fold_5.split(perchDF)

for index, (train, test) in enumerate(datasets):
    print(f'{index} -> {train} {test}')     # 출력되는 숫자들은 해당하는 데이터의 행번호

0 -> [12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55] [ 0  1  2  3  4  5  6  7  8  9 10 11]
1 -> [ 0  1  2  3  4  5  6  7  8  9 10 11 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55] [12 13 14 15 16 17 18 19 20 21 22]
2 -> [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55] [23 24 25 26 27 28 29 30 31 32 33]
3 -> [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 45 46 47 48 49 50 51 52 53 54 55] [34 35 36 37 38 39 40 41 42 43 44]
4 -> [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44] [45 46 47 48 49 50 51 52 53 54 55]


In [7]:
## 타겟이 분류인 경우
irisDF = pd.read_csv('../DATA/iris.csv')
irisDF.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [51]:
k_fold = KFold(n_splits=3, shuffle=True)
ret = k_fold.split(irisDF[irisDF.columns[:-1]])

In [52]:
from sklearn.linear_model import LogisticRegression

train_score, test_score = [], []

for idx, (train, test) in enumerate(ret):
    # 학습용, 테스트용 인덱스 반환
    train_idx = train.tolist()
    test_idx = test.tolist()
    
    # 인덱스에 해당하는 데이터 셋 추출
    trainDF = irisDF.iloc[train_idx]
    testDF = irisDF.iloc[test_idx]
    
    # print(trainDF.iloc[:, 4].value_counts() / trainDF.shape[0])
    # print(testDF.iloc[:, 4].value_counts() / testDF.shape[0])
    
    x_train = trainDF[trainDF.columns[:-1]]
    y_train = trainDF[trainDF.columns[-1]]
    
    x_test = testDF[testDF.columns[:-1]]
    y_test = testDF[testDF.columns[-1]]
    
    # 분류 모델 학습
    log_model = LogisticRegression(max_iter=1000, solver='liblinear')   # penalty 파라미터 예시 ['l1', 'l2', 'elasticnet']
    log_model.fit(x_train, y_train)
    
    # 훈련 및 검증용 성능 점수 저장
    train_score.append(log_model.score(x_train, y_train))
    test_score.append(log_model.score(x_test, y_test))
    
    # 예측
    # pred_y = log_model.predict(x_test)

In [53]:
def mean(some_list: list):
    if len(some_list) == 0:
        print('빈 리스트')
    else:
        return sum(some_list) / len(some_list)

In [54]:
mean(train_score), mean(test_score)

(0.96, 0.96)

In [55]:
train_score

[0.97, 0.96, 0.95]